In [ ]:
# -*- coding: utf-8 -*-
import os
import __init__

from ontomap.base import BaseConfig
from ontomap.ontology import ontology_matching
from ontomap.utils import io

root_dataset_dir = "../datasets"

statistics = {}

config = BaseConfig(root_dataset_dir=root_dataset_dir).get_args()
for ontology, oms in ontology_matching.items():
    print(f"working on {ontology} track OM pairs!")
    for om in oms:
        print(f"\t {om.ontology_name} pairs is processing!")
        dataset = om().collect(root_dir=config.root_dir)
        if dataset['dataset-info']['track'] not in statistics:
            statistics[dataset['dataset-info']['track']] = []
        if dataset['dataset-info']['track'] == 'bio-llm':
            ref_len = len(dataset['reference']['test-cands'])
        elif dataset['dataset-info']['track'] == 'bio-ml':
            ref_len = len(dataset['reference']['equiv']['full'])+len(dataset['reference']['subs']['test-cands'])+len(dataset['reference']['subs']['train'])
        else:
            ref_len = len(dataset['reference'])
        statistics[dataset['dataset-info']['track']].append({
            "task": dataset['dataset-info']['ontology-name'],
            "source": len(dataset['source']), 
            "target": len(dataset['target']),
            "reference": ref_len
        })

In [5]:
for track, track_items in statistics.items():
    print(track, end="")
    for task in track_items:
        print("\t", "\t", task['task'], "\t", task['source'], "\t", task['target'], "\t", task['reference'])
        # print()
    print("-"*90)

anatomy	 	 mouse-human 	 2737 	 3298 	 1516
------------------------------------------------------------------------------------------
food	 	 ciqual-siren 	 2807 	 19886 	 178
------------------------------------------------------------------------------------------
biodiv	 	 envo-sweet 	 6566 	 4525 	 805
	 	 fish-zooplankton 	 145 	 56 	 15
	 	 macroalgae-macrozoobenthos 	 108 	 128 	 18
	 	 taxrefldBacteria-ncbitaxonBacteria 	 312 	 326 	 175
	 	 taxrefldChromista-ncbitaxonChromista 	 2290 	 2344 	 1405
	 	 taxrefldFungi-ncbitaxonFungi 	 12732 	 13149 	 10162
	 	 taxrefldPlantae-ncbitaxonPlantae 	 26302 	 27013 	 19914
	 	 taxrefldProtozoa-ncbitaxonProtozoa 	 501 	 538 	 357
------------------------------------------------------------------------------------------
phenotype	 	 doid-ordo 	 15509 	 13504 	 1237
	 	 hp-mp 	 40703 	 37965 	 696
------------------------------------------------------------------------------------------
commonkg	 	 nell-dbpedia 	 134 	 137 	 129
	 	 yago-

In [6]:
io.write_json(output_path=os.path.join("../experiments/stats", "tracks-tasks-stats.json"),
              json_data=statistics)